In [1]:
import pandas as pd
titanic=pd.read_csv("train.csv",encoding='Latin-1')

In [2]:
print(titanic.shape)
print(titanic.head())
header=titanic.columns.tolist()
print(header)

(891, 12)
   PassengerId  Survived  Pclass  \
0            1         0       3   
1            2         1       1   
2            3         1       3   
3            4         1       1   
4            5         0       3   

                                                Name     Sex   Age  SibSp  \
0                            Braund, Mr. Owen Harris    male  22.0      1   
1  Cumings, Mrs. John Bradley (Florence Briggs Th...  female  38.0      1   
2                             Heikkinen, Miss. Laina  female  26.0      0   
3       Futrelle, Mrs. Jacques Heath (Lily May Peel)  female  35.0      1   
4                           Allen, Mr. William Henry    male  35.0      0   

   Parch            Ticket     Fare Cabin Embarked  
0      0         A/5 21171   7.2500   NaN        S  
1      0          PC 17599  71.2833   C85        C  
2      0  STON/O2. 3101282   7.9250   NaN        S  
3      0            113803  53.1000  C123        S  
4      0            373450   8.0500   NaN    

In [3]:
titanic.describe()

,PassengerId,Survived,Pclass,Age,SibSp,Parch,Fare
count,891.000000,891.000000,891.000000,714.000000,891.000000,891.000000,891.000000
mean,446.000000,0.383838,2.308642,29.699118,0.523008,0.381594,32.204208
std,257.353842,0.486592,0.836071,14.526497,1.102743,0.806057,49.693429
min,1.000000,0.000000,1.000000,0.420000,0.000000,0.000000,0.000000
25%,223.500000,0.000000,2.000000,20.125000,0.000000,0.000000,7.910400
50%,446.000000,0.000000,3.000000,28.000000,0.000000,0.000000,14.454200
75%,668.500000,1.000000,3.000000,38.000000,1.000000,0.000000,31.000000
max,891.000000,1.000000,3.000000,80.000000,8.000000,6.000000,512.329200


The count for Age has 714 which is less than other columns, which means we need to deal with missing value in Age column.

In [4]:
titanic['Age']=titanic['Age'].fillna(titanic['Age'].median())

In [5]:
titanic['Cabin'].isnull().sum() #count nan values in a column

687

In [6]:
#Convert Sex column to numeric
## df.loc[row_indexer,column_indexer] #df
## s.loc[indexer] #series
print(titanic['Sex'].unique())
titanic.loc[titanic['Sex']=='male',"Sex"]=0
titanic.loc[titanic['Sex']=='female',"Sex"]=1
#must have "Sex" as col index, SQUARE bracket .loc[]


['male' 'female']


In [7]:
titanic['Embarked'].value_counts() #S is the most frequent value
titanic['Embarked'].fillna('S',inplace=True)
# S-0, C-1, Q-2
titanic.loc[titanic['Embarked']=='S','Embarked']=0
titanic.loc[titanic['Embarked']=='C','Embarked']=1
titanic.loc[titanic['Embarked']=='Q','Embarked']=2

start machine learning part. Age V.S. Survival, linear Regression shortage: only tells you absolate value,not probability.
Overfitting is what happens when a model fits itself to "noise," rather than signal.
cross-validation can avoid overfitting. To cross-validate, we split our data into some number of parts, or "folds."
Combine the first two parts, train a model, and make predictions on the third.
Combine the first and third parts, train a model, and make predictions on the second.
Combine the second and third parts, train a model, and make predictions on the first.

In [8]:
# Import the linear regression class
from sklearn.linear_model import LinearRegression
# Sklearn also has a helper that makes it easy to do cross-validation
from sklearn.model_selection import KFold

# The columns we'll use to predict the target
predictors = ["Pclass", "Sex", "Age", "SibSp", "Parch", "Fare", "Embarked"]

# Initialize our algorithm class
alg = LinearRegression()
# Generate cross-validation folds for the titanic data set
# It returns the row indices corresponding to train and test
# We set random_state to ensure we get the same splits every time we run this
kf = KFold(3, random_state=1)

predictions = [] #an array
for train, test in kf.split(titanic): #test is an array of number 
        # The predictors we're using to train the algorithm  
    # Note how we only take the rows in the train folds
    train_predictors = (titanic[predictors].iloc[train,:])
    # The target we're using to train the algorithm
    train_target = titanic["Survived"].iloc[train]
    # Training the algorithm using the predictors and target
    alg.fit(train_predictors, train_target)
    # We can now make predictions on the test fold
    test_predictions = alg.predict(titanic[predictors].iloc[test,:])
    predictions.append(test_predictions)

In [9]:
#print(predictions)

In [10]:
#the error metric is the percentage of correct predictions. 
import numpy as np

# The predictions are in three separate NumPy arrays  
# Concatenate them into a single array, along the axis 0 (the only 1 axis) 
predictions = np.concatenate(predictions,axis=0)

In [11]:
# Map predictions to outcomes (the only possible outcomes are 1 and 0)
predictions[predictions > .5] = 1
predictions[predictions <=.5] = 0

In [12]:
#predictions==titanic['Survived'] #boolean value
accuracy=(sum(predictions==titanic['Survived']))/len(titanic)

Logistic regression takes the output of a linear regression and maps it to a probability value between 0 and 1. It does the mapping using the logit function. Passing any value through the logit function will map it to a value between 0 and 1 by "squeezing" the extreme values. This is perfect for us, because we only care about two outcomes.

In [13]:
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import cross_val_score

# Initialize our algorithm
alg = LogisticRegression(random_state=1)
# Compute the accuracy score for all the cross-validation folds; this is much simpler than what we did before
scores = cross_val_score(alg, titanic[predictors], titanic["Survived"], cv=3)
# Take the mean of the scores (because we have one for each fold)
print(scores.mean())

0.787878787879


In [14]:
#performing the exact same steps on the test data that we did on the training data. 
#If we don't perform the exact same operations, then we won't be able to make valid predictions on it.
titanic_test = pd.read_csv("test.csv")
titanic_test['Age'].fillna(titanic['Age'].median(),inplace=True)
#The age has to be the exact same value we used to replace the missing ages in the training set 
#it can't be the median of the test set, because this is different
titanic_test.loc[titanic_test['Sex']=='male','Sex']=0
titanic_test.loc[titanic_test['Sex']=='female','Sex']=1
titanic_test['Embarked'].fillna('S',inplace=True)
titanic_test.loc[titanic_test['Embarked']=='S','Embarked']=0
titanic_test.loc[titanic_test['Embarked']=='C','Embarked']=1
titanic_test.loc[titanic_test['Embarked']=='Q','Embarked']=2
titanic_test['Fare'].fillna(titanic_test['Fare'].median(),inplace=True)

In [15]:
#First, we have to train an algorithm on the training data. 
#Then, we'll make predictions on the test set. 
#Finally, we'll generate a CSV file with the predictions and passenger IDs.

# Initialize the algorithm class
alg = LogisticRegression(random_state=1)

# Train the algorithm using all the training data
alg.fit(titanic[predictors], titanic["Survived"])

# Make predictions using the test set
predictions = alg.predict(titanic_test[predictors])

# Create a new dataframe with only the columns Kaggle wants from the data set
submission = pd.DataFrame({
        "PassengerId": titanic_test["PassengerId"],
        "Survived": predictions
    })

In [16]:
submission.to_csv("kaggle.csv", index=False) #prediction has two cols, PassageID, 0/1